In [9]:
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation

import matplotlib.pyplot as plt
import seaborn as sns
import math
import pandas as pd
import numpy as np
from os.path import join
import os
from tqdm import tqdm

Using TensorFlow backend.
/anaconda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Read in hourly volume datasets

In [14]:
%%time
input_dir = join("data", "citibike_departures")
input_filename = "citibike_departures_lags.csv"
departures_by_neighborhood_lagged = pd.read_csv(join(input_dir, input_filename), index_col=0)

CPU times: user 27.4 s, sys: 1.6 s, total: 29 s
Wall time: 29.2 s


In [17]:
departures_by_neighborhood_lagged.shape

(10752, 8788)

In [15]:
%%time
input_dir = join("data", "citibike_arrivals")
input_filename = "citibike_arrivals_lags.csv"
arrivals_by_neighborhood_lagged = pd.read_csv(join(input_dir, input_filename), index_col=0)

CPU times: user 28.6 s, sys: 1.73 s, total: 30.3 s
Wall time: 30.5 s


In [16]:
arrivals_by_neighborhood_lagged.shape

(10752, 8788)

# Model # 1: Forecast Hourly Departures

In [20]:
departures_by_neighborhood_lagged.head()

,BK09,BK09_t-1,BK09_t-2,BK09_t-3,BK09_t-4,BK09_t-5,BK09_t-6,BK09_t-7,BK09_t-8,BK09_t-9,...,QN99_t-159,QN99_t-160,QN99_t-161,QN99_t-162,QN99_t-163,QN99_t-164,QN99_t-165,QN99_t-166,QN99_t-167,QN99_t-168
2017-01-08 00:00:00,2,1.0,0.0,2.0,1.0,0.0,2.0,2.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-08 01:00:00,0,2.0,1.0,0.0,2.0,1.0,0.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-08 02:00:00,0,0.0,2.0,1.0,0.0,2.0,1.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-08 03:00:00,0,0.0,0.0,2.0,1.0,0.0,2.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-08 04:00:00,0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# Feature variables
feature_columns = [i for i in departures_by_neighborhood_lagged.columns if '_t-' in i]
X = departures_by_neighborhood_lagged[feature_columns]

# Target variables
target_columns = [i for i in departures_by_neighborhood_lagged.columns if '_t-' not in i]
y = departures_by_neighborhood_lagged[target_columns]

# Reshape to arrays
X = np.array(X)

num_targets = len(target_columns)
y = np.array(y).reshape((len(y), num_targets))

In [33]:
X.shape

(10752, 8736)

In [34]:
y.shape

(10752, 52)

### Split into training and testing data

In [35]:
n_train_rows = 365 * 24

X_train = X[0 : n_train_rows, ]
X_test = X[n_train_rows:, ]

y_train = y[0 : n_train_rows]
y_test = y[n_train_rows:]

In [36]:
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.fit_transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.fit_transform(y_test)

# Reshape
X_train = X_train.reshape(X_train.shape + (1,))
X_test = X_test.reshape(X_test.shape + (1,))

In [37]:
X_train.shape

(8760, 8736, 1)

In [38]:
y_train.shape

(8760, 52)

In [39]:
X_test.shape

(1992, 8736, 1)

In [40]:
y_test.shape

(1992, 52)

### LSTM

In [41]:
seed = 7
np.random.seed(seed)

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
#model.add(Dense(100, activation='linear'))
#model.add(LSTM(100))
model.add(Dense(num_targets, activation='linear'))

model.compile(loss='mae', optimizer='adam')

# fit network
history = model.fit(X_train, y_train, epochs=10, batch_size=500,
                    validation_data=(X_test, y_test), verbose=2, shuffle=False)

Train on 8760 samples, validate on 1992 samples
Epoch 1/10


In [ ]:
history_loss = pd.DataFrame()
history_loss['train'] = history.history['loss']
history_loss['test'] = history.history['val_loss']
history_loss.plot(figsize=(10,10), fontsize=16,
                title='LSTM Model Loss');

In [ ]:
yhat = model.predict(X_test)
yhat = scaler_y.inverse_transform(np.array(yhat).reshape((len(yhat), num_targets)))
actual = scaler_y.inverse_transform(np.array(y_test).reshape((len(y_test), num_targets)))

In [ ]:
results = pd.DataFrame()

for i in range(num_targets+1):
    prediction_colname = 'pred{}'.format(i)
    actual_colname = 'actual{}'.format(i)
    results[prediction_colname] = yhat[:i]
    results[actual_colname] = actual[:i]